# Entrega Tarea 2
#### Estudiantes:
                    Carlos López 201830033-0
                    Marias Perez 

# Definicion del entorno de trabajo

# Prueba de coneccion con el cliente

El siguiente codigo activa una mision simple en la que el jugador avanza y salta para comprobar correcta coneccion con el cliente
adicionalmente se define las caracteristica de un mundo plano y la colocacion de bloques especificos por medio de coordenadas

# Dependencias

In [13]:
import sys, time
import MalmoPython as Malmo

print(f"Python version: {sys.version}")
print("MalmoPython importado correctamente")

agent_host = Malmo.AgentHost()

missionXML = '''<?xml version="1.0" encoding="UTF-8" standalone="no" ?>
<Mission xmlns="http://ProjectMalmo.microsoft.com" 
         xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">
  <About>
    <Summary>Prueba Malmo</Summary>
  </About>

  <ServerSection>
    <ServerInitialConditions>
      <Time>
        <StartTime>1000</StartTime>
        <AllowPassageOfTime>true</AllowPassageOfTime>
      </Time>
    </ServerInitialConditions>
    <ServerHandlers>
      <FlatWorldGenerator generatorString="3;7,2;1;"/>
      <DrawingDecorator>
        <DrawBlock x="-2" y="5" z="0" type="diamond_block"/>
      </DrawingDecorator>
      <ServerQuitWhenAnyAgentFinishes/>
    </ServerHandlers>
  </ServerSection>

  <AgentSection mode="Survival">
    <Name>CarlosBot</Name>
    <AgentStart>
      <Placement x="0.5" y="4" z="0.5" yaw="90"/>
    </AgentStart>
    <AgentHandlers>
      <ObservationFromFullStats/>
      <ContinuousMovementCommands/>
      <AgentQuitFromTouchingBlockType>
        <Block type="diamond_block"/>
      </AgentQuitFromTouchingBlockType>
    </AgentHandlers>
  </AgentSection>
</Mission>
'''

mission = Malmo.MissionSpec(missionXML, True)
mission_record = Malmo.MissionRecordSpec()

# 🔹 Indicar el cliente (IP de Windows)
client_pool = Malmo.ClientPool()
client_pool.add(Malmo.ClientInfo("127.0.0.1", 10001))  # Cambia por tu IP real

print("Iniciando misión...")
agent_host.startMission(mission, client_pool, mission_record, 0, "CarlosBot")

print("Esperando cliente...")
world_state = agent_host.getWorldState()
#print("Tipo de world_state:", type(world_state))
#print("Atributos:", dir(world_state))

while not world_state.has_mission_begun:
    print(".", end="")
    time.sleep(0.1)
    world_state = agent_host.getWorldState()
    for error in world_state.errors:
        print("Error:", error.text)
print("\n¡Misión iniciada! 🚀")


for _ in range(50):
    agent_host.sendCommand("move 1")
    agent_host.sendCommand("jump 1")
    time.sleep(0.1)
agent_host.sendCommand("move 0")
agent_host.sendCommand("jump 0")
print("Tipo de world_state:", type(world_state))
print("Atributos:", dir(world_state))
print("Misión terminada.")


Python version: 3.6.15 (default, Sep 30 2025, 22:53:39) 
[GCC 11.4.0]
MalmoPython importado correctamente
Iniciando misión...
Esperando cliente...
.................................
¡Misión iniciada! 🚀
Tipo de world_state: <class 'MalmoPython.WorldState'>
Atributos: ['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'errors', 'has_mission_begun', 'is_mission_running', 'mission_control_messages', 'number_of_observations_since_last_state', 'number_of_rewards_since_last_state', 'number_of_video_frames_since_last_state', 'observations', 'rewards', 'video_frames']
Misión terminada.


In [14]:
import sys, time
import MalmoPython as Malmo
from pathlib import Path
from string import Template
import re

# Comprobar versiones

In [15]:
print(f"Python version: {sys.version}")

# Ruta al archivo malmo.properties (ajústala si es necesario)
ruta = "/home/carlos/MalmoPlatform/Malmo-0.37.0-Linux-Ubuntu-18.04-64bit_withBoost_Python3.6/malmo.properties"

with open(ruta, "r") as f:
    linea = f.readline().strip()
# Extrae solo el número después del signo '='
version = linea.split("=")[-1]
print("Versión de Malmo:", version)

Python version: 3.6.15 (default, Sep 30 2025, 22:53:39) 
[GCC 11.4.0]
Versión de Malmo: 0.37.0


# Definiendo la mision de la ejecucion

Se definira las mision segun contenido de archivos xml

In [16]:
from typing import List  # compatible con 3.6–3.8

# Catálogo de misiones basado en archivos .xml
ROOT = Path().resolve()
MISSIONS_DIR = ROOT / "misiones"

def list_missions() -> List[str]:
    if not MISSIONS_DIR.exists():
        print(f"No existe la carpeta: {MISSIONS_DIR}")
        return []
    files = sorted(p.stem for p in MISSIONS_DIR.glob("*.xml"))
    print("Misiones disponibles:", ", ".join(files) if files else "(ninguna)")
    return files

def _load_xml(name: str) -> str:
    path = MISSIONS_DIR / f"{name}.xml"  # corregido
    if not path.exists():
        raise FileNotFoundError(f"No se encontró la misión: {path}")
    return path.read_text(encoding="utf-8")

def render_mission(name: str, **params) -> str:
    raw = _load_xml(name)
    converted = re.sub(r"\{\{(\w+)\}\}", r"${\1}", raw)
    return Template(converted).substitute(**params)

def missions_catalog(**defaults):
    def get(name: str, **overrides):
        return render_mission(name, **{**defaults, **overrides})
    return get

# Mostrar las misiones detectadas
list_missions()

# Instanciar el catálogo con valores por defecto y construir el XML
get_mission = missions_catalog(start_time=1000, agent_name="CarlosBot")



Misiones disponibles: busca_aldea, busca_carbon, busca_hierro, busca_madera, busca_piedra, flat_test, mundo_normal


# Escoger mision

In [19]:
missionXML = get_mission("flat_test")

# Realizando coneccion

In [20]:
agent_host = Malmo.AgentHost()
mission = Malmo.MissionSpec(missionXML, True)
mission_record = Malmo.MissionRecordSpec()

# 🔹 Indicar el cliente (IP de Windows)
client_pool = Malmo.ClientPool()
client_pool.add(Malmo.ClientInfo("127.0.0.1", 10001))  # Cambia por tu IP real

print("Iniciando misión...")
agent_host.startMission(mission, client_pool, mission_record, 0, "CarlosBot2")

print("Esperando cliente...")
world_state = agent_host.getWorldState()
#print("Tipo de world_state:", type(world_state))
#print("Atributos:", dir(world_state))

while not world_state.has_mission_begun:
    print(".", end="")
    time.sleep(0.1)
    world_state = agent_host.getWorldState()
    for error in world_state.errors:
        print("Error:", error.text)
print("\n¡Misión iniciada! 🚀")

Iniciando misión...
Esperando cliente...
.....
¡Misión iniciada! 🚀


# Instrucciones de movimiento

In [21]:
for _ in range(50):
    agent_host.sendCommand("move 1")
    agent_host.sendCommand("jump 1")
    time.sleep(0.1)

agent_host.sendCommand("move 0")
agent_host.sendCommand("jump 0")
print("Misión terminada.")

Misión terminada.


# Datos que se obtienen del entorno

In [22]:
print("Tipo de world_state:", type(world_state))
print("Atributos:", dir(world_state))

Tipo de world_state: <class 'MalmoPython.WorldState'>
Atributos: ['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'errors', 'has_mission_begun', 'is_mission_running', 'mission_control_messages', 'number_of_observations_since_last_state', 'number_of_rewards_since_last_state', 'number_of_video_frames_since_last_state', 'observations', 'rewards', 'video_frames']
